In [1]:
clean_up = True # if True, remove all gams related files from working folder before starting
%run stdPackages.ipynb
os.chdir(d['py'])
from gmsPython import nestingTree
from valueShares import nestedShares_noOutputs
from mHousehold import StaticConsumer

The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb1.gdx is still active and was not deleted.
The file _gams_py_gdb2.gdx is still active and was not deleted.
The file _gams_py_gdb3.gdx is still active and was not deleted.
The file _gams_py_gdb4.gdx is still active and was not deleted.


# Household module

*Load data:*

In [2]:
t0 = 2019 # set baseline year
name = 'vA' # specify name of the CGE version we are currently working on
db = GpyDB(os.path.join(d['data'], f'{name}_{t0}')) # Load 
ws = db.ws # run everything from the same ws

## 1. Nesting tree and value shares

Nesting tree:

In [3]:
cesNest = nestingTree.Tree('HH', tree = db('nestHH').to_list(), f = 'CES')
nest = nestingTree.AggTree(name = 'HH', trees = {t.name: t for t in [cesNest]}, ws = ws)
nest(namespace = {str(n)+'_input': n for n in db('n')})

Value shares:

In [4]:
v = nestedShares_noOutputs(nest, ws = ws)
db_vs = v(db) # one-liner that returns solution database

We are particularly interested in the $\mu$ variable and value of intermediate goods (to get better initial values):

In [5]:
db['mu'] = db_vs('mu').xs(t0) 
db.aom(adj.rc_pd(db_vs('vD'), nest.get('int').union(nest.get('output'))).rename('qD'), name = 'qD', priority = 'first') # specify intermediate goods levels

## 2. Partial Equilibirum Model

Only keep household sectors, extrapolate data for initial values:

In [6]:
AggDB.subsetDB(db, db('s_HH'))
[db.series.__delitem__(k) for k in ('vD','vTax', 'vD_dur','vD_depr','vD_inv', 'vS', 'pD_dur') if k in db.symbols];

For variables that are defined over $t$, but we do not yet have an initial value for all $t$, extrapolate:

*Note: This forces extrapolation of all variables defined over $t$ - if it is important that some variables are not extrapolated, they should be removed from this statement.*

In [7]:
[symbol.__setattr__('vals', extrapolateUpper(symbol.vals, db('tE')[0])) for symbol in [db[k] for k in db.varDom('t')['t']]];

Initialize module from nesting tree and add the mapping from labor supply to the consumption aggregate:

In [8]:
m = StaticConsumer(nest, partial = True, L2C = db('L2C'))
m.initStuff(db = db)

Use $j$-terms solution approach for calibration:

In [9]:
soldb = m.jSolve(3, state = 'C', ϕ = .1) # solve calibration model with 5 steps and nonlinear grid (ϕ<1 means that adjustments to jTerms start large and then decrease)

Overwrite solution:

In [10]:
[m.db.__setitem__(k, soldb[k]) for k in m.db.getTypes(['var']) if k in soldb.symbols]; # use solution database
m.db.mergeInternal()
m.solve(state = 'B') # test solve of baseline

## 3. Export

Baseline model text with fix/unfix and solve statements:

In [11]:
baselineText = m.write(state = 'B') # store text so we can export it for later
text_gamY    = m.write_gamY(state = 'B') # store gamY text

Export code and database:

In [12]:
with open(os.path.join(d['gams'], f'{m.name}.gms'), "w") as file:
    file.write(baselineText)
with open(os.path.join(d['gams'], f'{m.name}.gmy'), "w") as file:
    file.write(text_gamY)

Export gdx file:

In [13]:
m.db.database.export(os.path.join(d['data'], m.db.name))